# Deep Word Meaning
Do the meanings of words used in the political discourse change over time, can we quantify this? But this time we will be using a BERT GoEmotions model.

In [ ]:
import pandas as pd
import os
import sys
import json
import re

# from transformers import BertTokenizer
# from model import BertForMultiLabelClassification
# from multilabel_pipeline import MultiLabelPipeline
from fuzzysearch import find_near_matches
import re
from tqdm import tqdm
import numpy as np 
import pandas as pd
import pickle
import swifter

In [ ]:
import os
import sys
import pickle
from tqdm import tqdm
import itertools
import dateparser
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import matplotlib.dates as mdates

# Add repo
git_dir = os.path.abspath('../')

# Define data path
data_path = os.path.join(git_dir, 'data', 'pm-transcripts')\

# Define data output path
data_output_path = os.path.join(git_dir, 'data', 'pm-transcripts-processed')

# Pickle path
df_pickle_path_ekman = os.path.join(data_output_path, 'aggregated-spellfixed-typed-emotions-ekman.pkl')
df_pickle_path_emotion = os.path.join(data_output_path, 'aggregated-spellfixed-typed-emotions.pkl')

fig_dir = os.path.join(git_dir, 'figs')

#Read in, make string lists into strings
df_ekman = pd.read_pickle(df_pickle_path_ekman)
df_emotion = pd.read_pickle(df_pickle_path_emotion)

In [ ]:
# Drop useless columns and set a datetime index.
def clean_df(df):
    df = df.drop('Unnamed: 0', axis=1)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').set_index('date')
    df[df['date']>datetime.datetime(year=1970, month=1, day=1)]
    return df 

df_ekman = clean_df(df_ekman)
df_emotion = clean_df(df_emotion)

In [ ]:
df_ekman

In [ ]:
def vectorise_emotion(emotion, em_map):
    arr = np.zeros(len(em_map))
    for idx, em in enumerate(emotion['labels']):
        arr[em_map[em]] = emotion['scores'][idx]
    return arr

def fuzzy_colocation(sequences, search_key):
    ret = []
    for index, sequence in enumerate(sequences):
        if any([len(find_near_matches(subsequence, sequence, max_l_dist=2)) > 0 
             for subsequence in search_key]):
            ret.append(index)
        # Return neighbour indicies?
    return ret

def generate_keyword_in_context_sentiment(
    df_pass, # Copied after pass, new df is returned.
    keyword_tag, # Id for the dataframe
    keywords, # List of words, see how sentiment for this class changes over time.
    verbose = True
):
    if verbose: print("Copying df")
    df = df_pass.copy()

    if verbose: print("Starting fuzzy matching")
    df[f'{keyword_tag}_idx'] = df['sentences'].swifter\
    .apply(lambda x: fuzzy_colocation(x, keywords))
    
    # For each document we now have a list of sentences that express some emotion
    # Average over them.
    def calculate_mean_em(row):
        emotions_size = len(row['emotions_vecs'])
        return np.mean([row['emotions_vecs'][idx] 
                   for idx in row[f'{keyword_tag}_idx'] 
                   if idx < emotions_size], axis=0).tolist()
    
    if verbose: print("Calculating mean emotion per document per keyword")
    df[f'{keyword_tag}'] = df.swifter.apply(calculate_mean_em, axis = 1)
    return df

def plot_keyword_dep(
    df,
    keyword_tag,
    emotion_map
):
    view = df[f'{keyword_tag}'].apply(pd.Series)
    view = view.rename(columns = lambda col: emotion_map[col])

    plt.figure()
    view.plot()
    plt.figure()
    view_rolling = view.rolling('120d').mean()
    view_rolling.plot()

In [ ]:
ekman_list = ['anger', 
              'disgust', 
              'fear', 
              'joy', 
              'sadness', 
              'surprise', 
              'neutral']
ekman_map = dict(zip(ekman_list, range(len(ekman_list))))

positive_emotions = ['admiration', 
                     'amusement',
                     'approval',
                     'caring',
                     'desire',
                     'excitement',
                     'gratitude',
                     'joy',
                     'love',
                     'optimism',
                     'pride',
                     'relief']
negative_emotions = ['anger',
                     'annoyance',
                     'disappointment',
                     'disapproval',
                     'disgust',
                     'embarrassment',
                     'fear',
                     'grief',
                     'nervousness',
                     'remorse',
                     'sadness']
ambiguous_emotions = ['confusion',
                      'curiosity',
                      'realization',
                      'surprise']
neutral_emotions = ['neutral']

emotions_list = positive_emotions + negative_emotions + ambiguous_emotions + neutral_emotions
emotions_map = dict(zip(emotions_list, range(len(emotions_list))))

In [ ]:
# Pad emotion into a vector 
df_ekman[f'emotions_vecs'] = df_ekman['sentences_emotions'].swifter\
    .apply(lambda emotions: [vectorise_emotion(emotion, ekman_map) 
                             for emotion in emotions])

df_emotion[f'emotions_vecs'] = df_emotion['sentences_emotions'].swifter\
    .apply(lambda emotions: [vectorise_emotion(emotion, emotions_map) 
                             for emotion in emotions])

In [ ]:
woi = {
    'ind': ['aboriginal', 'first nations', 'aborigine', 'indigenous', 'torres strait island'],
    'econ': ['economy', 'economic'],
    'aus': ['australia', 'australian', 'aussie', 'australians'],
    'gov': ['government']
}

In [ ]:
df = generate_keyword_in_context_sentiment(df_ekman, 'aus', woi['aus'])

In [ ]:
plot_keyword_dep(df, 'aus', list(ekman_map.keys()))